In [15]:
# datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'2025-04-11 18:28:55'

In [ ]:
from datetime import datetime
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import gspread
import re
# GET -Получить данные со страници.
# POST -Передать данные или авторизацию.

header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}
page = 1
max_page = 1
data = []

while page <= max_page:

    link = (f'https://baza-knig.rip/fantastika-fenteze/page/{page}/')

    response = requests.get(link, headers=header)
    response.encoding = 'utf-8'  # Устанавливаем правильную кодировку

    soup = BeautifulSoup(response.text, 'html.parser')
    block = soup.find('div', id="dle-content")


    # Ищем все элементы с книгами
    books = block.find_all('div', class_='short')


    for book in books:
        book_data = {}

        # Извлекаем заголовок книги
        title = book.find('div', class_='short-title').text.strip()
        book_data["Название"] = title

        # Обрабатываем мета-данные книги
        for li in book.find('ul', class_='reset short-items').find_all('li'):
            parts = li.text.strip().split(':', 1) # Делим по первому вхождению ":
            if len(parts) == 2:
                key, value = parts
                book_data[key] = value

        data.append(book_data)
    page += 1


    # Эта конструкция помодет избавиться о Многоточей в страницах. И продолжеть Нумерацию страниц.
    button_navigation = soup.find('div', class_='bottom-nav ignore-select')
    pages = [p.text.strip() for p in button_navigation.find('div', class_="navigation")]
    int_pages = []
    for p in pages:
        try:
            n = int(p)
            int_pages.append(n)
        except:
            continue


# Удаление ненужныз символов.
patterns = {
    r"Фантастика": "Фан",
    r"фэнтези": "фэн",
    r",": "/",
    r" ": ""
}

for item in data:
    genre = item["Жанр"]
    for pattern, replacement in patterns.items():
        genre = re.sub(pattern, replacement, genre, flags=re.IGNORECASE)
    item["Жанр"] = genre




# df = pd.DataFrame(data)
df = pd.DataFrame(data).fillna("нет данных")
df['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
# df.head()


# print(len(data))


In [18]:
gc = gspread.service_account(filename='creds.json')
wks = gc.open("web_parsing_baza_knig").sheet1
wks.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1b3GmBL5gA5px_BdnN37sSslnGlQgrnDqX-jiQTtKSVI',
 'updatedRange': "'Лист1'!A1:H12",
 'updatedRows': 12,
 'updatedColumns': 8,
 'updatedCells': 96}

In [10]:
status_kicke = []
for item in block.find_all('div', class_="short-bottom"):
    like_dislike = {}
    res = item.text.strip().split("\n")
    # like_dislike['like-dis-comm'] = f'like = {res[1]}/dis = {res[2]}/comment = {res[3]}'
    like_dislike['like-dis-comm'] = f'like = {res[1]}/dis = {res[2]}/comment = {res[3]}'
    status_kicke.append(like_dislike)

# log["print"] =
# df[status_kicke[0].keys()] = status_kicke
# df

[dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm'])]

In [3]:
import gspread

gc = gspread.service_account(filename='creds.json')

# Open a sheet from a spreadsheet in one go
wks = gc.open("web_parsing_baza_knig").sheet1

# Update a range of cells using the top left corner address
wks.update([[1, 2], [3, 4]], 'A1')

# Or update a single cell
wks.update_acell('B42', "it's down there somewhere, let me take another look.")

# Format the header
wks.format('A1:B1', {'textFormat': {'bold': True}})

{'spreadsheetId': '1b3GmBL5gA5px_BdnN37sSslnGlQgrnDqX-jiQTtKSVI',
 'replies': [{}]}